# Veri Bilimi Temel Proje Örneği

Titanic veri seti üzerinden temel veri işleme, veri analizi ve analitiği üzerine bir notebook hazırladık. Titanic veri seti veri bilimine yeni başlayanlar için temel işlemleri anlama açısından önemli bir çıkış noktasıdır. 

Peki veri bilimci olarak nasıl bir çalışma stratejisi belirlemeliyiz. 
Çoğu zaman veriyi alıp çeşitli keşif analizleri yapılabilir.
Bu keşifler sonucunda çalışmanın genel hatlarını oluşturulup başlayabilirsiniz.
Ancak hangi aşamada olursa olsun sormamız gereken sorular vardır. 
Biz bu çalışma için, veriyi hiç görmeden temel bir hat oluşturacağız


1. Problem tanımı, bizden ne isteniyor?
2. Eğitim ve Test datalarımız mevcut mu?(Bu kursun kapsamında değil ancak yoksa veri toplama sürecine geçeriz)
3. Veriyi keşfet, temizle, analiz için hazırla.
4. İlk analizlerini oluştur, kuracağın model için veri dönüşümlerini yap.
5. Modeli oluştur, test et, problem sonucuna uygun cevaplar mı alıyorsun? (Hayır ise modeli ve modeli besleyen analizi
tekrar gözden geçir)
6. Sonuçları görselleştir, raporla, bulduğun en etkin çözümü sun.
7. Kullandığın veri, kütüphane, sistem versiyonları ve modeli detaylı anlatan bir çözüm kitapcığı hazırla.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

İlk olarak veri işleme, temizleme, hazırlama ve görselleştirme kütüphanelerini indirelim. Bu temel python eğitimi değildir. Python güçlü kütüphaneleri olan, veri bilimi ve yapay zeka uygulamalarında oldukça popüler bir programlama dilidir. Python temelleri için diğer eğitimlere bakmanızda fayda var. 

Genel bir kabul olarak kodlamada kullancağımız değişken ve fonksiyon isimleri kendilerini açıkayacak isimlerde olursa tekrar okunabilirliği artar. 

In [ ]:
train_df = pd.read_csv("../input/titanic/train.csv")

In [ ]:
train_df.head(10)

In [ ]:
train_df.shape

.shape önemili bir method. veri biliminde işlem yaptığımız verisetinin boyutları çok öenmlidir. özellikle ileride makine öğrenmesi modelleri kullanırken, veri boyutlarının tutarlı olması gerekir. 

In [ ]:
train_df.columns.values

Categorical : Survived , Sex, Embarked. 
Ordinal Category : PClass
Numerical : Age, Fare. 
Discrete Numerical : SibSp, Parch.

In [ ]:
train_df.Sex.value_counts()

.value_counts() fonksiyonu birçok defa kullanacağımız veri setinin değişkenkleri/değişkenine ait değerleri grup bazında toplar. 

In [ ]:
train_df.Pclass.value_counts()

Bu işlemi tüm değişkenler için mi yapmamız gerekiyor? Veri görselleştirme işte tam bu işe yarıyor. Çok büyük veri setlerinde hem temel işlemler ile hem de görselleştirme araçları ile veri keşfi yapılır. 

In [ ]:
train_df.info()

python .info() fonksiyonu veri setimizle ilgili temel bilgileri hızlıca bize verir. örneğin memory usage, non-null (boş olmayan) gözlem sayısı veri tipleri gibi. 

In [ ]:
train_df.isnull().sum()

.isnull() ve .sum() yine python un hazır fonksiyonları ve bize her zaman kullanabileceğimiz bir kalıp sunuyor. boş gözlemlerin değişken bazında toplam değeri. 

In [ ]:
train_df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']].value_counts()

Birden çok değişkene uygulanabilir ancak çok takip edilebilir bir sonuç değil.

In [ ]:
train_df.describe()

.describe() fonksiyonu veri setindeki değişkenlerin istatistiki bilgilerinşi bize verir, toplam sayı, ortalama, standart sapma min/max değerleri ile quantil değerleri. Bu fonksiyon sadece numeric değişkenlerin istatiksel verilerini verir, character tipi olanları değerlenmeye almaz. 

In [ ]:
train_df.describe(include=["O"])

Yukarıda info() fonksiyonuna bakarken bazı değişkenlerin object olduğunu görmüştük. include="O" methodu object tipi değişkenleri özetlemek için 

Yukarıdaki ilk veri incelemelerine göre bazı çıkarımlarımız olacak. 
1. Cabin / Age / Embarked verilerinde kayıp vei bulunmakta
2. Name değişkeni 891/891 unique değerde
3. Cabin ve Ticket değişkenlerinde duplicate veriler bulunmakta.

Bir veri analizine bakmadan önce "domain" bilgisine sahip olmak çok öenmlidir. Bu kazada kayıpların çoğunluğu
yeterli tahliye kayığı olmadığı için yaşanmış. Tahliye kayıklarına kadınlar/çocuklar ve üst sınıf yolcuların erişiminin daha çok olduğu varsayabiliriz. Üst sınıf yolcular Pclass da 1 indexli, Kadınlar Sex değişkeninde "female" indexli, çocuklarında yaş verilerinden çıkartılabilir. 

In [ ]:
#pivoting variables
train_df[["Sex","Survived"]].groupby(["Sex"], as_index = False).mean()

.groupby() pivot oluşturmak için kullanılan, bir 

In [ ]:
train_df[["Pclass","Survived"]].groupby(["Pclass"], as_index = False).mean()

In [ ]:
train_df[["Age","Survived"]].groupby(["Age"], as_index = False).mean().sort_values(by="Survived",ascending = False)

In [ ]:
age_plot = sns.FacetGrid(train_df,col="Survived")
age_plot.map(plt.hist,"Age",bins = 25)

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x="Pclass",y="Age",data=train_df,hue="Survived")

In [ ]:
pd.cut(train_df["Age"],bins=[0,14,60,100],include_lowest=True,labels=["child","adult","elder"])

In [ ]:
train_df["age_group"] = pd.cut(train_df["Age"],bins=[0,14,60,100],include_lowest=True,labels=["child","adult","elder"])

In [ ]:
train_df.head(20)

In [ ]:
sns.catplot(x="Pclass",y="Survived",col="age_group",data=train_df,kind="bar")

In [ ]:
train_df.age_group.value_counts()

In [ ]:
train_df[train_df["age_group"] == "child"].Pclass.value_counts()

In [ ]:
train_df[(train_df["age_group"] == "child") & (train_df["Survived"] == 1) ].Pclass.value_counts()

In [ ]:
train_df[train_df["age_group"] == "adult"].Pclass.value_counts()

In [ ]:
train_df[(train_df["age_group"] == "adult") & (train_df["Survived"] == 1) ].Pclass.value_counts()

## 4C => Correct - Complete - Create - Convert

In [ ]:
#before move on 
test_df = pd.read_csv("../input/titanic/test.csv")

In [ ]:
test_df

In [ ]:
test_df.isnull().sum()

In [ ]:
train_df.drop("Cabin",axis=1)

In [ ]:
train_df["family_size"] = train_df.SibSp + train_df.Parch + 1

In [ ]:
train_df.head(20)

düzeltme için hızlıca verilere bakıp bir yazım yanlışı var mı, çok anlamsız bir sayısal veri var mı diye bakabiliriz. yukarıda ki genel veri incelemelerinde göze batan en büyük şey eksik verilerin olduğu değişkenlerdi. bundan sonra veri tamamlama yapabiliriz, ardından yüksek korelasyonlu değişkenleri çıkartarak modelin daha doğru tahminleme yapması için veriyi hazırlarız. Son olarak, verilerimizi makine öğrenmesi için daha anlamlı olan veri yapılarına çeviririz. 

In [ ]:
# so lets start from the scracth.
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
# daha evvelden yaptığımız keşifleri düzenli olarak işleme alacagiz
#daha evvelki araştırmamızda bazı sütunları atacağımızı söylemiştik
train.drop("Cabin",axis=1,inplace=True)
test.drop("Cabin",axis=1,inplace=True)
#also pax id ve name bizim için belirleyici değişkenler değil. 
train.drop(["PassengerId","Name"],axis=1,inplace=True)
test.drop(["PassengerId","Name"],axis=1,inplace=True)

In [ ]:
#peki bilet numarası? bilet numarası için tekrar bakalım
print(train.Ticket.value_counts())
train.Ticket.isnull().sum()

In [ ]:
#cinsiyetler doğru mu acaba? baze male yerine Male yahut m yazılabilir. 
train.Sex.value_counts()

In [ ]:
#düzeltmeler bitti, şimdi doldurmalar. 
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
#hedef belli oldu. Age :)
train.Age.describe()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x="Sex",y="Age",data=train,hue="Survived")

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x="Sex",y="Age",data=test)

In [ ]:
#as we can see mean values for survived or not, male or female at 30. so we can impute 30.0
train["Age"].fillna(train["Age"].mean(),inplace=True)
test["Age"].fillna(test["Age"].mean(),inplace=True)

In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())

In [ ]:
print(train.Embarked.value_counts())
print(train.Embarked.isnull().sum())
print(test.Embarked.value_counts())
print(test.Embarked.isnull().sum())

In [ ]:
train["Embarked"].fillna("S",inplace=True)
test["Fare"].fillna(test["Fare"].mean(),inplace=True)

In [ ]:
#time to create! şimdi yeni değişkenler yaratma zamanı. 
#buarada veriyi inceledikten sonra veri bilimci değişkenler hakkında bir öngörüye sahip olur.
#iyi bir veri bilimci kendisine verilenler ile yetinmez, yaratır. (Create)

In [ ]:
train["family_size"] = train["SibSp"] + train["Parch"] + 1
test["family_size"] = test["SibSp"] + test["Parch"] + 1

In [ ]:
train["age_group"] = pd.cut(train["Age"],bins=[0,14,60,100],include_lowest=True,labels=["child","adult","elder"])
test["age_group"] = pd.cut(test["Age"],bins=[0,14,60,100],include_lowest=True,labels=["child","adult","elder"])

In [ ]:
# child, adult, elder açıklayıcılık anlamında çok iyi ancak ileride öğreneceğimiz gibi,
# numeric kodlama 0,1,2 çok daha uygun olacaktır. böylece bir çok model için tekrar dönüşüm gerekmez. 


In [ ]:
# peki sibsp ve parch yerine sadece family size kullanabilir miyiz?
train.SibSp.value_counts().plot(kind="bar")

In [ ]:
train.Parch.value_counts().plot(kind="bar")

In [ ]:
train.family_size.value_counts().plot(kind="bar")

In [ ]:
#yukarıda görebileceğimiz gibi sadece family_size değişkeni ile oransallığı bozmadan değişkenlerimizi ifade ediyoruz.
train.drop(["SibSp","Parch"],axis=1,inplace=True)
test.drop(["SibSp","Parch"],axis=1,inplace=True)

In [ ]:
train.drop("Ticket",axis=1,inplace=True)
test.drop("Ticket",axis=1,inplace=True)

In [ ]:
train.rename(columns={"Sex": "gender","Pclass": "class","Embarked":"station","Age": "age"},inplace=True)
test.rename(columns={"Sex": "gender","Pclass": "class","Embarked":"station","Age": "age"},inplace=True)

In [ ]:
# şimdi dönüşüme hazırız. 

In [ ]:
train["fare_group"] = pd.cut(train["Fare"],10,include_lowest=True)
test["fare_group"] = pd.cut(train["Fare"],10,include_lowest=True)

In [ ]:
test.fare_group.value_counts()

In [ ]:
train.fare_group.value_counts()

In [ ]:
train.Fare.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

LabelEncoder ve OneHotEncoder ne işe yarar, nasıl çalışır farkları nelerdir? Başka yöntemler var mıdır?

In [ ]:
train["age_code"] = label.fit_transform(train["age_group"])
train["fare_code"] =label.fit_transform(train["fare_group"])
train["gender_code"] = label.fit_transform(train["gender"])
train["station_code"] = label.fit_transform(train["station"])

In [ ]:
test["age_code"] = label.fit_transform(test["age_group"])
test["fare_code"] =label.fit_transform(test["fare_group"])
test["gender_code"] = label.fit_transform(test["gender"])
test["station_code"] = label.fit_transform(test["station"])

In [ ]:
# not alın :) neler yaptık
# gender M -> 1 F -> 0 oldu
# station S -> 2 Q->1 C->0
#age_group adult -> 0 child -> 1 elder -> 2
# fare aşağıda

In [ ]:
train.fare_code.value_counts()

In [ ]:
train.fare_group.value_counts()

In [ ]:
train.age_group.value_counts()

In [ ]:
train.age_code.value_counts()

In [ ]:
test.age_group.value_counts()

In [ ]:
test.age_group.isnull().sum()

In [ ]:
test.age.isnull().sum()

In [ ]:
test["age_code"] = label.fit_transform(test["age_group"])
test["fare_code"] =label.fit_transform(test["fare_group"])
test["gender_code"] = label.fit_transform(test["gender"])
test["station_code"] = label.fit_transform(test["station"])

In [ ]:
ftrain = train.drop(["gender","Fare","station","age_group","fare_group","age"],axis=1)

In [ ]:
ftrain.head(5)

In [ ]:
ftest = test.drop(["gender","Fare","station","age_group","fare_group","age"],axis=1)

In [ ]:
ftest.head(5)

In [ ]:
#ready for take-off!!!

In [ ]:
y = ftrain["Survived"]

In [ ]:
X = ftrain.drop(["Survived"],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=123)

In [ ]:
# aday algoritmalar ,i logistic regression ,decision tree, xgboost

machine learning algoritmalarının altında yatan matematiği bilmek özellikle bir model kurmadan önce önemlidir. bu notebook un konusu dahilinde değil ancak birçok açık kaynak bu algoritmaları anlatmaktadır. 
örneğin logistic regression için; 
https://machinelearningmastery.com/logistic-regression-for-machine-learning/
python un en çok kullanılan makine öğrenmesi kütüphanesi sklearn (scikit learn) logistic regression modulü için;
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

yukarıda label encoder için yaptığımız gibi LogisticRegression modulü import edildikten sonra önce classifier ı bir değişkene atarız. bu sadece kullanım açısında değil kodun okunabilirliği açısından ve işlemek açısından çok önemlidir. 

In [ ]:
logReg = LogisticRegression()

label encoder gibi classifier çeşitleri sadece modele veriyi işlemekle değil aynı zamanda bir transformasyon yapmayı hedeflediği için fit ve transfor fonksiyonlarını beraber gerçekleştiren fit_transform fonksiyonunu kullanır. ancak logistic regessin classifier'ı için sadece fit() fonksiyonun kullnamamız gerekir. 

In [ ]:
logReg.fit(X_train,y_train)

burada biraz duralım! fit() estimator fonksiyonu supervised learning modellerinde X -> sample y ->target verilerini alarak estimator fonksiyonuna işler. sadece bu iki veriyi aynı zamanda ağırlık serisi varsa sample_weight parametrelerini alarak ona göre veri desenini işleyebilir. biz default weight none olarak aldığı için buraya yazmayacağız. 

In [ ]:
y_pred = logReg.predict(X_test)

şimdi burada bir kez daha duralım, sklearn fit_predict() fonksiyonu mevcut, biz neden fit() ve predict() i ayrıca kullandık? fit() estimator ü ve predict() method u beraber aynı anda işlemek unsupervised learning için uygun bir method. bu tarz öğrenimde y -> none default değerinde olacaktır. 

In [ ]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logReg.score(X_test, y_test)))

In [ ]:
res = pd.DataFrame()
res["actual"] = y_test
res["pred"] = y_pred

In [ ]:
res.head(15)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
#buarada sonucu okumak çok önemli precision nedir? recall nedir f1-score nedir? accuracy nasıl hesaplanır.?


https://medium.com/@raghaviadoni/evaluation-metrics-i-precision-recall-and-f1-score-3ec25e9fb5d3

The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier to not label a sample as positive if it is negative.
The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.
The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.
The F-beta score weights the recall more than the precision by a factor of beta. beta = 1.0 means recall and precision are equally important.
The support is the number of occurrences of each class in y_test.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

In [ ]:
dt.fit(X_train,y_train)

In [ ]:
y_pred_dt = dt.predict(X_test)

In [ ]:
print('Accuracy of decision tree classifier on test set: {:.2f}'.format(dt.score(X_test, y_test)))

In [ ]:
print(classification_report(y_test,y_pred_dt))

In [ ]:
# biraz görselleştirelim mi?
import graphviz
from sklearn import tree

In [ ]:
dtc = dt.fit(X_train,y_train)

In [ ]:
tree.plot_tree(dtc)

In [ ]:
 
dtc_data = tree.export_graphviz(dtc, out_file=None) 
graph = graphviz.Source(dtc_data)

In [ ]:
graph

In [ ]:
feat_names = list(X_train.columns.values)

In [ ]:
dtc_data = tree.export_graphviz(dtc, out_file=None, feature_names=feat_names) 
graph = graphviz.Source(dtc_data)

In [ ]:
graph